Sjekker systemkrav

In [30]:
import subprocess
from typing import List

def execute_command(cmd: List[str], **kwargs):
    return subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        **kwargs
    )

system_requirements = [
    ["docker-compose", "--version"],
    ["docker", "--version"],
]

for requirement in system_requirements:
    assert execute_command(requirement, check=False).returncode == 0, f"System requirement {requirement} is not satisfied"

# PM Tiles raster data

Denne notebooken konverterer GeoTiff til PMTiles ved å bruke rio-pmtiles verktøyet.

## Input data

![nedlasting](../../public/img/geonorge-n50-raster-krs.png)

Last ned filen som vist over, pakk ut og flytt .tif filen til `data/N50_raster_2024.tif`



Først sjekker vi at input filen finnes som forventet.

In [31]:
import utils

INPUT_FILE = "N50_raster_2024.tif"
assert utils.path_exists(f"raster/data/{INPUT_FILE}"), f"Mangler inputfil {INPUT_FILE}"

Så bruker vi kommandolinjeverktøyet til å generere pmtiles filen.

In [32]:
!rio pmtiles raster/data/N50_raster_2024.tif raster/out/N50_raster_2024.pmtiles --format PNG

1185.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


100%|██████████████████████████████████████| 1425/1425 [00:11<00:00, 121.63it/s]


# PM Tiles vektor data